# Match passive properties

In [ ]:
import sys
sys.path.append("../../")

import numpy as np

from cell_builder import L5_seg_cell_builder

from act.cell_model import ACTCellModel
from act.simulator import ACTSimulator
from act.types import SimulationParameters, ConstantCurrentInjection
from act.passive import ACTPassiveModule
from act.types import SimulationParameters, ConstantCurrentInjection

## Get the soma / cell area

ACT asssumes soma parameters to be distributed (i.e., per unit of area). Due to that, we first run a short dummy simulation to find the soma area and the total cell area of our model.

In [ ]:
seg_cell = ACTCellModel(
    cell_name = None,
    path_to_hoc_file = None,
    path_to_mod_files = "../../data/L5/seg_tuned/modfiles/",
    passive = ["g_pas", "e_pas", "gbar_Ih"],
    active_channels = ["gbar_Nap", "gbar_K_T", "gbar_NaTa", "gbar_Kd", "gbar_Ca_LVA", "gbar_Ca_HVA", "gbar_Kv2like", "gbar_Kv3_1"]
)

seg_cell.set_custom_cell_builder(L5_seg_cell_builder)

sim_params = SimulationParameters(
    sim_name = "L5_seg", 
    sim_idx = 1, 
    h_celsius = 37,
    verbose = True, # Set verbose = True to print the area
    CI = [ConstantCurrentInjection(amp = -0.1, dur = 200, delay = 100)])

simulator = ACTSimulator(output_folder_name = "output")
simulator.submit_job(
    seg_cell,
    sim_params
)
simulator.run_jobs(1)


        ACTSimulator (2025)
        ----------
        When submitting multiple jobs, note that the cells must share modfiles.
        
Setting g_pas to 0.00083376 in soma
Setting e_pas to -60.501 in soma
Setting g_pas to 0.00928766 in axon
Setting e_pas to -61.4849 in axon
Setting g_pas to 0.00140526 in apic
Setting e_pas to -80.0882 in apic
Setting g_pas to 5.71849e-05 in dend
Setting e_pas to -65.1148 in dend
Setting cm to 3.56803 in soma
Setting Ra to 131.026 in soma
Setting cm to 6.72363 in axon
Setting Ra to 113.282 in axon
Setting cm to 9.89109 in apic
Setting Ra to 80.0605 in apic
Setting cm to 5.67721 in dend
Setting Ra to 88.3083 in dend
Adding mechanism K_T to soma
Setting gbar_K_T to 0.000540302 in soma
Adding mechanism Kd to soma
Setting gbar_Kd to 0.000951294 in soma
Adding mechanism Kv2like to soma
Setting gbar_Kv2like to 0.0686615 in soma
Adding mechanism NaTa to soma
Setting gbar_NaTa to 0.0173416 in soma
Adding mechanism Nap to soma
Setting gbar_Nap to 0.0173416 in s

In [579]:
computed_soma_area = 6.802158872916739e-06 # (cm2)
computed_total_area = 5.2523997815020434e-05 # (cm2)

## Set target passive properties

In [ ]:
target_rheobase = 190 # pA
target_tau = 26.6 * 1e-3 # (s)
target_R_in = 167 * 1e6 # Ohm
target_V_rest = -71.7 # mV

I = [150, 190, 230, 270, 330]
F = [0, 7, 12, 15, 21]

## Get analytical estimates for passive properties

We use `ACTPassiveModule` to compute settable passive properties (i.e., those that can be directly set in the model's files) from the target passive properties. We recommend using the soma area for simplified cells and the total area for detailed cells and scaling the area (without changing the actual cell area) until a reasonable `Cm` value is obtained. E.g., below we had to scale the total area by 3 to obtain a good estimate for `Cm`.

In [581]:
ACTPassiveModule.compute_spp(target_R_in, computed_total_area * 3, target_tau, target_V_rest)

SettablePassiveProperties(Cm=1.0108486517388842, g_bar_leak=3.8001829012740004e-05, e_rev_leak=-71.7, g_bar_h=None)

Now we manually update the model's files with these estimates. We recommend using the same values for each compartment.

## Iteratively fine-tune passive properties

Finally, we simulate the updated model providing a negative current injection and comptue the gettable passive properties from the simulated passive traces. We then manually adjust `g-pas` and `Cm` until a sufficient match with the target. In the example below we multiplied `g-pas` by 4.5 since the initial input resistance was too high.

In [ ]:
# Make g-pas higher
3.8001829012740004e-05 * 4.5

In [858]:
seg_cell = ACTCellModel(
    cell_name = None,
    path_to_hoc_file = None,
    path_to_mod_files = "../../data/L5/seg_tuned/modfiles/",
    passive = ["g_pas", "e_pas", "gbar_Ih"],
    active_channels = ["gbar_Nap", "gbar_K_T", "gbar_NaTa", "gbar_Kd", "gbar_Ca_LVA", "gbar_Ca_HVA", "gbar_Kv2like", "gbar_Kv3_1"]
)

seg_cell.set_custom_cell_builder(L5_seg_cell_builder)

sim_params = SimulationParameters(
    sim_name = "L5_seg", 
    sim_idx = 1, 
    h_celsius = 37,
    verbose = False,
    h_tstop = 1000,
    CI = [ConstantCurrentInjection(amp = -0.1, dur = 700, delay = 100)])

simulator = ACTSimulator(output_folder_name = "output")
simulator.submit_job(
    seg_cell,
    sim_params
)
simulator.run_jobs(1)


        ACTSimulator (2025)
        ----------
        When submitting multiple jobs, note that the cells must share modfiles.
        
Setting g_pas to 0.000171008 in soma
Setting e_pas to -71.7 in soma
Setting g_pas to 0.000171008 in axon
Setting e_pas to -71.7 in axon
Setting g_pas to 0.000171008 in apic
Setting e_pas to -71.7 in apic
Setting g_pas to 0.000171008 in dend
Setting e_pas to -71.7 in dend
Setting cm to 1.01085 in soma
Setting Ra to 131.026 in soma
Setting cm to 1.01085 in axon
Setting Ra to 113.282 in axon
Setting cm to 1.01085 in apic
Setting Ra to 80.0605 in apic
Setting cm to 1.01085 in dend
Setting Ra to 88.3083 in dend
Adding mechanism K_T to soma
Setting gbar_K_T to 0.000540302 in soma
Adding mechanism Kd to soma
Setting gbar_Kd to 0.000951294 in soma
Adding mechanism Kv2like to soma
Setting gbar_Kv2like to 0.0686615 in soma
Adding mechanism NaTa to soma
Setting gbar_NaTa to 0.0173416 in soma
Adding mechanism Nap to soma
Setting gbar_Nap to 0.0173416 in soma
Addi

In [859]:
# Get PP
passive_trace = np.load("output/L5_seg/out_1.npy")[:, 0]
seg_gpp = ACTPassiveModule.compute_gpp(passive_trace, dt = 0.1, I_t_start = 100, I_t_end = 700, I_amp = -0.1)
seg_gpp

GettablePassiveProperties(R_in=169.04155867370633, tau1=5.1000000000000005, tau2=70.10000000000001, sag_ratio=0.010980251266292403, V_rest=-71.90341766067336)